In [5]:
import onnx
from onnx import helper, mapping

# load your model
model = onnx.load("models/w600k_mbf.onnx")

# collect all initializer (weight) dtypes
dtypes = {
    mapping.TENSOR_TYPE_TO_NP_TYPE[t.data_type]
    for t in model.graph.initializer
}
print("Weight dtypes:", dtypes)

# also check all graph inputs and outputs
io_types = set()
for io in list(model.graph.input) + list(model.graph.output):
    t = io.type.tensor_type.elem_type
    io_types.add(mapping.TENSOR_TYPE_TO_NP_TYPE[t])
print("I/O dtypes:", io_types)

Weight dtypes: {dtype('float32')}
I/O dtypes: {dtype('float32')}


/var/folders/lg/ltf4rt5x5_7b6r3d1ty50m3h0000gq/T/ipykernel_33686/491560778.py:9: DeprecationWarning: `mapping.TENSOR_TYPE_TO_NP_TYPE` is now deprecated and will be removed in a future release.To silence this warning, please use `helper.tensor_dtype_to_np_dtype` instead.
  mapping.TENSOR_TYPE_TO_NP_TYPE[t.data_type]
/var/folders/lg/ltf4rt5x5_7b6r3d1ty50m3h0000gq/T/ipykernel_33686/491560778.py:18: DeprecationWarning: `mapping.TENSOR_TYPE_TO_NP_TYPE` is now deprecated and will be removed in a future release.To silence this warning, please use `helper.tensor_dtype_to_np_dtype` instead.
  io_types.add(mapping.TENSOR_TYPE_TO_NP_TYPE[t])


In [4]:
import onnxruntime as ort

sess = ort.InferenceSession("models/det_500m.onnx")
for inp in sess.get_inputs():
    print(f"Input `{inp.name}`: {inp.type}")   # e.g. "tensor(float)"
for out in sess.get_outputs():
    print(f"Output `{out.name}`: {out.type}")


Input `input.1`: tensor(float)
Output `443`: tensor(float)
Output `468`: tensor(float)
Output `493`: tensor(float)
Output `446`: tensor(float)
Output `471`: tensor(float)
Output `496`: tensor(float)
Output `449`: tensor(float)
Output `474`: tensor(float)
Output `499`: tensor(float)


In [7]:
import os
import random
import shutil

# 1. Update these paths to your source and target directories:
src_dir = '/Users/yousufsarfraz_1/Downloads/archive (4)/images/train'
dst_dir = '/Users/yousufsarfraz_1/Documents/GitHub/mds-25/downsizing_experimental/calibration_images_v2'

# Create target directory if it doesn't exist
os.makedirs(dst_dir, exist_ok=True)

# 2. Collect all .png files from the source directory
png_files = [f for f in os.listdir(src_dir)]

# 3. Randomly sample up to 400 files
sample_count = min(400, len(png_files))
sampled_files = random.sample(png_files, sample_count)

# 4. Copy each sampled file to the target directory
for filename in sampled_files:
    src_path = os.path.join(src_dir, filename)
    dst_path = os.path.join(dst_dir, filename)
    shutil.copy(src_path, dst_path)

print(f"Copied {sample_count} images from\n  {src_dir}\nto\n  {dst_dir}")


Copied 400 images from
  /Users/yousufsarfraz_1/Downloads/archive (4)/images/train
to
  /Users/yousufsarfraz_1/Documents/GitHub/mds-25/downsizing_experimental/calibration_images_v2


In [8]:
import os
import cv2
import numpy as np
import onnxruntime
from onnxruntime.quantization import quantize_static, CalibrationDataReader, QuantType

# Paths
model_input = "/Users/yousufsarfraz_1/.insightface/models/buffalo_sc/det_500m.onnx"
model_output = "/Users/yousufsarfraz_1/Documents/GitHub/mds-25/downsizing_experimental/det_500m.static_int8_v2.onnx"
calib_folder = "/Users/yousufsarfraz_1/Documents/GitHub/mds-25/downsizing_experimental/calibration_images_v2"

# 1. Create an ONNX Runtime session to fetch the input name
session = onnxruntime.InferenceSession(model_input, providers=["CPUExecutionProvider"])
input_name = session.get_inputs()[0].name

# 2. Calibration data reader
class FaceCalibReader(CalibrationDataReader):
    def __init__(self, image_folder, input_name):
        self.image_paths = [
            os.path.join(image_folder, f)
            for f in os.listdir(image_folder)
            if f.lower().endswith((".jpg", ".jpeg", ".png"))
        ]
        self.input_name = input_name
        self.iter = iter(self.image_paths)

    def get_next(self):
        try:
            img_path = next(self.iter)
            # Load image (BGR), resize, convert to RGB
            img = cv2.imread(img_path)
            img = cv2.resize(img, (640, 640))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            # Normalize to [0,1], transpose to (C,H,W), add batch dim
            img = img.astype(np.float32) / 255.0
            img = np.transpose(img, (2, 0, 1))[None, ...]
            return {self.input_name: img}
        except StopIteration:
            return None

calib_reader = FaceCalibReader(calib_folder, input_name)

# 3. Run static quantization
quantize_static(
    model_input=model_input,
    model_output=model_output,
    calibration_data_reader=calib_reader,
    activation_type=QuantType.QInt8,
    weight_type=QuantType.QInt8
)

print(f"Static INT8 quantized model saved to: {model_output}")


Static INT8 quantized model saved to: /Users/yousufsarfraz_1/Documents/GitHub/mds-25/downsizing_experimental/det_500m.static_int8_v2.onnx
